Reference: https://www.kaggle.com/code/kojimar/fb3-single-pytorch-model-train

In [1]:
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/MyDrive/kaggle/notebook

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/kaggle/notebook


In [2]:
%%capture
!pip install transformers==4.20.1
!pip install tokenizers==0.12.1
!pip install sentencepiece==0.1.97

In [3]:
import os
import gc
import time
import math
import datetime

import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import LambdaLR


import transformers
import tokenizers
from transformers import AutoModel, AutoTokenizer, AutoConfig
from transformers import get_cosine_schedule_with_warmup

os.system('pip install iterative-stratification==0.1.7')
os.environ['TOKENIZERS_PARALLELISM']='true'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
class CFG:
    model_name  = "microsoft/deberta-v3-base"
    
    base        = "../input/feedback-prize-english-language-learning/"
    train       = base + "train.csv"
    test        = base + "test.csv"
    target_cols = ['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']
    num_targets = 6
    
    # Backbone
    max_length  = 512
    hidden_dims = 768
    output_hidden_states = True
    pool = 'mean'
    reinit   = True
    reinit_n = 1
    
    # loss
    loss = 'L1smooth'
    apex = True
    max_norm = 1000
    
    # optimizer
    default_lr   = 1e-5
    head_lr      = 1e-4
    adam_eps     = 1e-6
    lr_decay     = 0.9
    weight_decay = 0.01
    
    # scheduler
    scheduler = 'lambda'
    lr_sch_decay_steps = 1600
    num_warmup_steps = 0.0
    num_cycles = 0.5
    
    # CV
    n_folds = 5
    seed   = 42
    epochs = 5
    
    # Loader
    batch_size  = 4
    num_workers = 2
    print_freq = 20

    str_now = datetime.datetime.now().strftime('%Y%m%d-%H%M%S')

In [5]:
if len(CFG.model_name.split("/")) == 2:
   CFG.identifier = f"{CFG.str_now}-{CFG.model_name.split('/')[1]}"
else:
   CFG.identifier = f"{CFG.str_now}-{CFG.model_name}"

CFG.OUTPUT_DIR = f'./{CFG.identifier}/'
CFG.log_file   = CFG.OUTPUT_DIR + 'train'
os.makedirs(CFG.OUTPUT_DIR, exist_ok = True)

In [6]:
df_train = pd.read_csv(CFG.train)

In [7]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [8]:
fold = MultilabelStratifiedKFold(n_splits = CFG.n_folds, shuffle = True, random_state = CFG.seed)
df   = df_train.copy()
y    = pd.get_dummies(data = df[CFG.target_cols], columns = CFG.target_cols)

for idx, (train_idx, valid_idx) in enumerate(fold.split(df_train, y)):
    df_train.loc[valid_idx, 'fold'] = idx

In [9]:
class AverageMeter(object):
    def __init__(self):
        self.reset()
             
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        
    def update(self, val, n = 1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return f'{int(m)}m {int(s)}s'
        
def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    
    return f'{str(asMinutes(s))} (remain {str(asMinutes(rs))})'
        
def prepare_inputs(texts):
    inputs = []
    for text in texts:
        tokens = tokenizer.encode_plus(text,
                                       add_special_tokens = True,
                                       max_length = CFG.max_length,
                                       pad_to_max_length = True,
                                       truncation = True,
                                       return_attention_mask = True
                                       )
        
        for key, vals in tokens.items():
            tokens[key] = torch.tensor(vals, dtype = torch.long)
        inputs.append(tokens)
    
    return inputs

class MeanPooling(nn.Module):
      def __init__(self):
          super(MeanPooling, self).__init__()

      def forward(self, inputs, attention_masks):
          #print("inputs.shape: ", inputs.shape, " masks.shape: ", attention_masks.shape)
          attention_masks = attention_masks.unsqueeze(-1).float()
          inputs   = torch.sum(inputs*attention_masks, dim = 1)
          num_masks = torch.sum(attention_masks == 1.0, dim = 1)
          torch.clamp(num_masks, min = 1e-9)
          inputs   = (inputs / num_masks)

          return inputs

class AttentionPooling(nn.Module):
      def __init__(self, hidden_size):
          super(AttentionPooling, self).__init__()
          self.attention = nn.Sequential(nn.Linear(hidden_size, hidden_size),
                                         nn.GELU(),
                                         nn.Linear(hidden_size, 1)
                                         )

      def forward(self, hidden_state, attention_mask):
          weights = self.attention(hidden_state)
          weights[attention_mask == 0] = float("-inf")
          weights = torch.softmax(weights, dim = 1)
          context = torch.sum(hidden_state * weights, dim = 1)

          return context

def MCRMSE(labels, preds):
    scores = []
    num_targets = labels.shape[1]
    
    for i in range(num_targets):
        pred  = preds[:,i]
        label = labels[:,i]
        score = mean_squared_error(label, pred, squared = False)
        scores.append(score)
    
    mcrmse = np.mean(scores)
    
    return mcrmse, scores 

def get_score(labels, preds):
    return MCRMSE(labels, preds)

def get_logger(log_file = CFG.log_file):
    from logging import getLogger, INFO, StreamHandler, FileHandler, Formatter
    logger = getLogger(__name__)
    logger.setLevel(INFO)
    handler1 = StreamHandler()
    handler1.setFormatter(Formatter('%(message)s'))
    handler2 = FileHandler(filename = f'{log_file}.log')
    handler2.setFormatter(Formatter('%(message)s'))
    logger.addHandler(handler1)
    logger.addHandler(handler2)
    logger.propagate = False
    
    return logger
        

In [10]:
LOGGER = get_logger()
LOGGER.info(f'OUTPUT_DIR: {CFG.OUTPUT_DIR}')

OUTPUT_DIR: ./20221213-000822-deberta-v3-base/


In [11]:
from tqdm import tqdm

tokenizer = AutoTokenizer.from_pretrained(CFG.model_name)
#tokenizer.add_tokens(["\n"], special_tokens=True)

CFG.max_len = 1429
LOGGER.info(f'max_len: {CFG.max_len}')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:434: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
max_len: 1428


In [12]:
class FB3Dataset(Dataset):
    def __init__(self, df_train):
        self.inputs = prepare_inputs(df_train['full_text'])
        self.labels = torch.tensor(df_train[CFG.target_cols].values, dtype = torch.float)
        
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        return self.inputs[idx], self.labels[idx]

In [13]:
class FB3Model(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.config     = AutoConfig.from_pretrained(cfg.model_name, output_hidden_states = cfg.output_hidden_states)
        self.config.hidden_dropout = 0.
        self.config.hidden_dropout_prob = 0.
        self.config.attention_dropout = 0.
        self.config.attention_probs_dropout_prob = 0.

        self.model      = AutoModel.from_pretrained(cfg.model_name, 
                                                    config = self.config)
        self.cfg        = cfg

        LOGGER.info(self.cfg)

        self.pool       = MeanPooling()
        self.classifier = nn.Linear(cfg.hidden_dims, cfg.num_targets)
        self._init_weights(self.classifier)

    def feature(self, inputs):
        outputs = self.model(**inputs)
        feature = self.pool(outputs.last_hidden_state, inputs['attention_mask'])
 
        return feature

    def _init_weights(self, module):
        module.weight.data.normal_(mean = 0.0, std = 0.02)

        if module.bias is not None:
            module.bias.data.zero_()

    def reinitialize_layer(self, layer_num):
        for layer in self.model.encoder.layer[-layer_num:].modules():
            if isinstance(layer, nn.Linear):
               layer.weight.data.normal_(mean = 0.0, std = 0.02)

               if layer.bias is not None:
                  layer.bias.data.zero_()

            elif isinstance(layer, nn.Embedding):
               layer.weight.data.normal_(mean = 0.0, std = 0.02)

               if layer.padding_idx is not None:
                  layer.weight.data[layer.padding_idx].zero_()

            elif isinstance(layer, nn.LayerNorm):
                layer.bias.data.zero_()
                layer.weight.data.fill_(1.0)
          
    def forward(self, inputs):

        feature = self.feature(inputs)
        outputs = self.classifier(feature)
        
        return outputs

In [14]:
def train_fn(model, criterion, optimizer, scheduler, train_loader, epoch, cfg):
    losses = AverageMeter()
    total_steps = len(train_loader)
    start = end = time.time()
    model.train()
    scaler = torch.cuda.amp.GradScaler(enabled=cfg.apex)
    
    for step, (inputs, labels) in enumerate(train_loader):
        #print("inputs.shape: ", inputs['input_ids'].shape)
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        
        with torch.cuda.amp.autocast(enabled = cfg.apex):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
        losses.update(loss.item(), labels.shape[0])
        
        scaler.scale(loss).backward()
        grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), cfg.max_norm)
          
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        scheduler.step()
        
        end = time.time()
        
        if (step + 1) % cfg.print_freq == 0:
            print("[{0}][{1}/{2}] "
                  "Elapsed: {remain:s} "
                  "Loss: {loss.val:.4f}({loss.avg:.4f}) "
                  "Grad: {grad_norm} "
                  "LR: {lr:.8f} "
                  .format(epoch, step, total_steps, remain = timeSince(start, float(step+1)/total_steps),
                          loss = losses,
                          grad_norm = grad_norm,
                          lr = scheduler.get_lr()[0]
                          )
                 )
    
    return losses.avg

def valid_fn(model, criterion, valid_loader, epoch, cfg):
    losses = AverageMeter()
    predictions = []
    total_steps = len(valid_loader) - 1
    model.eval()
    start = end = time.time()
    
    for step, (inputs, labels) in enumerate(valid_loader):
        
        for k, v in inputs.items():
            inputs[k] = v.to(device)
        labels = labels.to(device)
        
        with torch.no_grad():
            preds = model(inputs)
        
        loss = criterion(preds, labels)
        losses.update(loss.item(), labels.shape[0])
        
        predictions.append(preds.detach().cpu().numpy())
        
        if (step + 1) % cfg.print_freq == 0 or step == total_steps:
            print("[{0}][{1}/{2}] "
                  "Elapsed: {remain:s} "
                  "Loss: {loss.val:.4f}({loss.avg:.4f}) "
                  .format(epoch+1, step, total_steps, remain = timeSince(start, float(step+1)/total_steps),
                          loss = losses,
                          )
                 )
    
    predictions = np.concatenate(predictions, axis = 0)
    
    return losses.avg, predictions

In [15]:
from collections import defaultdict

def train_loop(folds, fold, cfg):
    LOGGER.info(f"========== fold: {fold} training ==========")
    train_folds  = folds[folds['fold'] != fold]
    valid_folds  = folds[folds['fold'] == fold]
    valid_labels = valid_folds[cfg.target_cols].values
    
    train_dataset = FB3Dataset(train_folds)
    valid_dataset = FB3Dataset(valid_folds)
    
    train_loader  = DataLoader(train_dataset, 
                               batch_size  = cfg.batch_size,
                               shuffle = True,
                               pin_memory = True,
                               num_workers = cfg.num_workers)
    
    valid_loader  = DataLoader(valid_dataset, 
                               batch_size = cfg.batch_size,
                               shuffle = False,
                               pin_memory = True,
                               num_workers = cfg.num_workers)
    
    def get_optimizer(cfg, model):
        
        all_parameters = list(model.named_parameters())
        used_name_parameters = set()
        params = []
        no_wd = ['word_embeddings', 'bias', 'LayerNorm.weight']

        head   = [(n, p) for n, p in model.named_parameters() if 'model' not in n]
        for n, _ in head: used_name_parameters.add(n)
  
        params.append({"params": [p for n, p in head if not any(no in n for no in no_wd)], "weight_decay": 0.01, "lr": cfg.head_lr})
        params.append({"params": [p for n, p in head if any(no in n for no in no_wd)], "weight_decay": 0.0, "lr": cfg.head_lr})
        
        lr = cfg.default_lr
        decay = 0.9

        layer_list = [model.model.embeddings] + list(model.model.encoder.layer)
        layer_list.reverse()

        params.append(
            {
                "params": list(model.model.encoder.rel_embeddings.parameters()),
                "lr": lr,
                "weight_decay": 0.0
            }
        )

        params.append(
            {
                "params": list(model.model.encoder.LayerNorm.parameters()),
                "lr": lr,
                "weight_decay": 0.0
            }
        )
      
        for i, layer in enumerate(layer_list):
            
            parameters = [(n, p) for n, p in layer.named_parameters()]
            for n, _ in parameters: used_name_parameters.add(n)

            params.append({"params": [p for n, p in parameters if not any(no in n for no in no_wd)], 'weight_decay': 0.01, "lr": lr})
            params.append({"params": [p for n, p in parameters if any(no in n for no in no_wd)], 'weight_decay': 0.0, "lr": lr})

            lr = lr * decay

        optimizer = torch.optim.AdamW(params, eps = cfg.adam_eps)

        return optimizer

    def get_scheduler(cfg, optimizer, num_train_steps):
        warmup_start = int(cfg.epochs * num_train_steps)
        print("warmup start: ", warmup_start)

        def decay_func(step):
            return 0.3 ** (float(step) / float(cfg.lr_sch_decay_steps))

        if cfg.scheduler == 'linear':
           scheduler = transformers.get_scheduler(cfg.scheduler, optimizer, warmup_start, num_train_steps)
        elif cfg.scheduler == 'cosine':
           scheduler = get_cosine_schedule_with_warmup(
               optimizer,
               num_warmup_steps = cfg.num_warmup_steps,
               num_training_steps = num_train_steps,
               num_cycles = cfg.num_cycles
           )
        elif cfg.scheduler == 'lambda':
           scheduler = LambdaLR(optimizer, lr_lambda = decay_func)

        return scheduler
    
    model = FB3Model(cfg).to(device)
    if CFG.reinit:
       model.reinitialize_layer(CFG.reinit_n)
    criterion = nn.SmoothL1Loss(reduction = 'mean')
    optimizer = get_optimizer(cfg, model)
    num_train_steps = int(len(train_folds) / cfg.batch_size * cfg.epochs)
    print("[DEBUG] num_train_steps: ", num_train_steps, " len train loader: ", len(train_loader))
    scheduler = get_scheduler(cfg, optimizer, num_train_steps)

    best_score = np.inf
    best_val_loss = np.inf
    best_train_loss = np.inf

    for epoch in range(cfg.epochs):
        
        start = time.time()
        
        avg_loss = train_fn(model, criterion, optimizer, scheduler, train_loader, epoch, cfg)
        
        avg_val_loss, preds = valid_fn(model, criterion, valid_loader, epoch, cfg)
        score, scores = get_score(preds, valid_labels)
        
        elapsed = time.time() - start
        
        LOGGER.info(f"Epoch: {epoch+1} - avg_train_loss = {avg_loss:.4f} avg_val_loss: {avg_val_loss:.4f} time: {elapsed:.4f}s")
        LOGGER.info(f"Epoch: {epoch+1} - Score: {score:.4f}, Scores: {scores}")

        if best_score > score:
           best_score = score
           best_train_loss = avg_loss
           bset_val_loss = avg_val_loss
           LOGGER.info(f'Epoch {epoch+1} - Save Best Score: {best_score:.4f} Model')

           torch.save({'model': model.state_dict(),
                       'predictions': preds},
                      CFG.OUTPUT_DIR + f"{CFG.model_name.replace('/', '-')}_fold{fold}_best.pth")
    
    predictions = torch.load(CFG.OUTPUT_DIR + f"{CFG.model_name.replace('/', '-')}_fold{fold}_best.pth", 
                             map_location = torch.device('cpu'))['predictions']
    valid_folds[[f"pred_{c}" for c in CFG.target_cols]] = predictions

    torch.cuda.empty_cache()
    gc.collect()

    return best_train_loss, best_val_loss, valid_folds, best_score

In [16]:
def get_result(oof_df, fold, best_train_loss, best_val_loss):
    labels = oof_df[CFG.target_cols].values
    preds = oof_df[[f"pred_{c}" for c in CFG.target_cols]].values
    score, scores = get_score(labels, preds)
    LOGGER.info(f'Score: {score:<.4f}  Scores: {scores}')

    return None

oof_df = pd.DataFrame()
train_loss_list = []
val_loss_list = []

for fold in range(CFG.n_folds):
    best_train_loss, best_val_loss, _oof_df, best_score = train_loop(df_train, fold, CFG)
    oof_df = pd.concat([oof_df, _oof_df])
    train_loss_list.append(best_train_loss)
    val_loss_list.append(best_val_loss)

    LOGGER.info(f"========== fold: {fold} result ==========")

    get_result(_oof_df, fold, best_train_loss, best_val_loss)

oof_df = oof_df.reset_index(drop = True)
LOGGER.info(f"========== CV ==========")
get_result(oof_df, "OOF", np.mean(train_loss_list), np.mean(val_loss_list))
oof_df.to_csv(CFG.OUTPUT_DIR + 'oof_df.csv')

========== fold: 0 training ==========
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2301: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2Model: ['mask_predictions.dense.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.bi

[DEBUG] num_train_steps:  3912  len train loader:  783
warmup start:  19560


/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


[0][19/783] Elapsed: 0m 7s (remain 4m 53s) Loss: 0.3765(1.5168) Grad: 246857.6875 LR: 0.00009851 
[0][39/783] Elapsed: 0m 14s (remain 4m 24s) Loss: 0.2607(0.9170) Grad: 88933.359375 LR: 0.00009703 
[0][59/783] Elapsed: 0m 20s (remain 4m 10s) Loss: 0.1563(0.6692) Grad: 186682.390625 LR: 0.00009559 
[0][79/783] Elapsed: 0m 27s (remain 4m 0s) Loss: 0.0779(0.5441) Grad: 141349.421875 LR: 0.00009416 
[0][99/783] Elapsed: 0m 33s (remain 3m 51s) Loss: 0.0964(0.4679) Grad: 80614.7578125 LR: 0.00009275 
[0][119/783] Elapsed: 0m 40s (remain 3m 44s) Loss: 0.2041(0.4099) Grad: 165196.90625 LR: 0.00009137 
[0][139/783] Elapsed: 0m 47s (remain 3m 37s) Loss: 0.1572(0.3700) Grad: 154415.484375 LR: 0.00009000 
[0][159/783] Elapsed: 0m 53s (remain 3m 30s) Loss: 0.1772(0.3404) Grad: 227742.5 LR: 0.00008866 
[0][179/783] Elapsed: 1m 0s (remain 3m 22s) Loss: 0.0742(0.3158) Grad: 98222.4296875 LR: 0.00008733 
[0][199/783] Elapsed: 1m 7s (remain 3m 15s) Loss: 0.1501(0.2959) Grad: 117917.921875 LR: 0.00008603

Epoch: 1 - avg_train_loss = 0.1631 avg_val_loss: 0.1074 time: 300.7663s
Epoch: 1 - Score: 0.4651, Scores: [0.48553763825112545, 0.46286388668305734, 0.43294210874559136, 0.46273514523289316, 0.47782467958336794, 0.46845511347539165]
Epoch 1 - Save Best Score: 0.4651 Model


...

/usr/local/lib/python3.8/dist-packages/torch/optim/lr_scheduler.py:257: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "


[4][19/784] Elapsed: 0m 6s (remain 4m 14s) Loss: 0.0925(0.0964) Grad: 200985.03125 LR: 0.00000930 
[4][39/784] Elapsed: 0m 13s (remain 4m 5s) Loss: 0.0773(0.0922) Grad: 82997.375 LR: 0.00000916 
[4][59/784] Elapsed: 0m 19s (remain 3m 58s) Loss: 0.0655(0.0924) Grad: 116106.9921875 LR: 0.00000903 
[4][79/784] Elapsed: 0m 26s (remain 3m 51s) Loss: 0.0605(0.0913) Grad: 61096.40234375 LR: 0.00000889 
[4][99/784] Elapsed: 0m 32s (remain 3m 45s) Loss: 0.0519(0.0920) Grad: 85458.9140625 LR: 0.00000876 
[4][119/784] Elapsed: 0m 39s (remain 3m 38s) Loss: 0.0801(0.0924) Grad: 189732.9375 LR: 0.00000863 
[4][139/784] Elapsed: 0m 46s (remain 3m 32s) Loss: 0.1062(0.0908) Grad: 148996.5625 LR: 0.00000850 
[4][159/784] Elapsed: 0m 52s (remain 3m 25s) Loss: 0.0471(0.0893) Grad: 80307.5546875 LR: 0.00000837 
[4][179/784] Elapsed: 0m 59s (remain 3m 18s) Loss: 0.0547(0.0886) Grad: 72383.2578125 LR: 0.00000825 
[4][199/784] Elapsed: 1m 5s (remain 3m 12s) Loss: 0.0653(0.0882) Grad: 87686.078125 LR: 0.000008

Epoch: 5 - avg_train_loss = 0.0933 avg_val_loss: 0.1049 time: 299.1705s
Epoch: 5 - Score: 0.4587, Scores: [0.4854830223515189, 0.4438361967028614, 0.41789888625461663, 0.46396889856107, 0.48186613262388917, 0.45892999541854645]
Epoch 5 - Save Best Score: 0.4587 Model


[5][194/194] Elapsed: 0m 41s (remain -1m 59s) Loss: 0.0867(0.1049) 


/usr/local/lib/python3.8/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
========== fold: 4 result ==========
Score: 0.4587  Scores: [0.4854830223515189, 0.4438361967028614, 0.41789888625461663, 0.46396889856107, 0.48186613262388917, 0.45892999541854645]
========== CV ==========
Score: 0.4550  Scores: [0.4843101272329265, 0.4469884731288726, 0.41886618468616554, 0.4594054979227242, 0.4713665400029151, 0.4492003125655974]
